In [9]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

# CGV

## Event List

In [44]:
def get_CGV_GoodsIdx(event_idx):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayEventDetail"
    body = {
        "eventIndex": event_idx,
        "giveawayIndex": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    goods_idx = loads["d"]["GiveawayItemList"][0]["GiveawayItemCode"]
    return goods_idx

def get_CGV_EventList(movie_filter='', goods_filter=''):
    url = "https://m.cgv.co.kr/Event/GiveawayEventList.aspx/GetGiveawayEventList"
    body = {
        "theaterCode": "",
        "pageNumber": "1",
        "pageSize": "100"
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    soup = BeautifulSoup(loads['d']['List'])
    event_list = soup.find_all("li")
    event_df = pd.DataFrame()
    p = re.compile(r'\[([\d\D]+)\]') # 영화이름 추출 정규식
    for event in event_list:
        event_idx = event['data-eventidx']
        goods_idx = get_CGV_GoodsIdx(event_idx)
        event_name = event.find("strong").get_text()
        event_period = event.find("span").get_text()
        m = p.match(event_name)
        if m is None:
            movie_name = event_name
            goods_name = event_name
        else:
            movie_name = m.group()[1:-1]
            goods_name = event_name[m.end():] # 띄어쓰기 했다안했다
        event_dict = {"event_idx":event_idx, "goods_idx":goods_idx, "movie":movie_name, "goods":goods_name, "period":event_period}
        event_df = pd.concat([event_df, pd.DataFrame.from_dict(event_dict, orient='index').T], ignore_index=True)
    event_df = event_df[
        (event_df["movie"].str.contains(movie_filter)) &
        (event_df["goods"].str.contains(goods_filter))
    ]
    # 날짜 필터도 추가 예정
    # 위에서 필터링
    return event_df

In [45]:
event_df = get_CGV_EventList()
event_df

,event_idx,goods_idx,movie,goods,period
0,202404083924,2024040839243921,골드핑거,필름마크,2024.04.10 ~ 2024.05.09
1,202405034019,2024050340194022,"낸 골딘,~",프리미어DAY 오리지널 포스터,2024.05.04 ~ 2024.05.06
2,202404304009,2024043040094012,로봇 드림,포스터 2종 세트,2024.05.01 ~ 2024.05.07
3,202404173975,2024041739753979,범죄도시4,4DX포스터,2024.04.24 ~ 2024.05.22
4,202404173974,2024041739743978,범죄도시4,IMAX포스터,2024.04.24 ~ 2024.05.22
5,202404304011,2024043040114014,범죄도시4,백창기 티저 포스터,2024.05.01 ~ 2024.05.07
6,202404223980,2024042239803984,범죄도시4,필름마크,2024.04.24 ~ 2024.05.23
7,202404033915,2024040339153906,슈가│어거스트 디 투어~,IMAX 포스터,2024.04.10 ~ 2024.05.08
8,202404254005,2024042540054008,슈가│어거스트 디 투어~,4주차 렌티큘러카드,2024.05.01 ~ 2024.05.07
9,202404233984,2024042339843988,스턴트맨,IMAX포스터,2024.05.01 ~ 2024.05.29


## Goods

- type2: 재고보유가능성높음
- type3: 재고소진임박
- type4: 마감

In [55]:
def get_CGV_Theaters(giveawayIndex):
    url = "https://m.cgv.co.kr/Event/GiveawayEventDetail.aspx/GetGiveawayTheaterInfo"
    body = {
        "giveawayIndex": giveawayIndex,
        "areaCode": None
    }
    response = requests.post(url, json=body)
    text = response.text
    loads = json.loads(text)
    AreaList = loads["d"]["AreaList"]
    Theaters = pd.DataFrame()
    for Area in AreaList:
        AreaCode = Area["AreaCode"]
        body = {
            "giveawayIndex": giveawayIndex,
             "areaCode": AreaCode
        }
        response = requests.post(url, json=body)
        text = response.text
        loads = json.loads(text)
        Theater = pd.DataFrame(loads["d"]["TheaterList"])
        Theaters = pd.concat([Theaters, Theater[["TheaterName", "TheaterCode", "CountTypeCode"]]], ignore_index=True)
    return Theaters

In [56]:
giveawayIndex = '2024042940064009'
Theaters = get_CGV_Theaters(giveawayIndex)
Theaters

,TheaterName,TheaterCode,CountTypeCode
0,CGV 강변,0001,type2
1,CGV 명동역 씨네라이브러리,0105,type4
2,CGV 신촌아트레온,0150,type2
3,CGV 압구정,0040,type4
4,CGV 여의도,0112,type4
5,CGV 용산아이파크몰,0013,type4
6,CGV 오리,0004,type2
7,CGV 인천,0002,type4
8,CGV 춘천,0070,type3
9,CGV 대전,0007,type4


In [57]:
def Get_CGV_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_CGV_EventList(movie_filter=movie_filter, goods_filter=goods_filter)
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        giveawayIndex = event_df.loc[idx]["goods_idx"]
        Theaters = get_CGV_Theaters(giveawayIndex)
        Theaters["movie"] =  event_df.loc[idx]["movie"]
        Theaters["goods"] =  event_df.loc[idx]["goods"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [58]:
GoodsStatus = Get_CGV_GoodsStatus(movie_filter="쿵푸팬더", goods_filter="TTT")
GoodsStatus

,TheaterName,TheaterCode,CountTypeCode,movie,goods,period
0,CGV 강남,0056,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
1,CGV 강변,0001,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
2,CGV 구로,0010,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
3,CGV 동대문,0252,type3,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
4,CGV 등촌,0230,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
...,...,...,...,...,...,...
58,CGV 서전주,0225,type2,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
59,CGV 전주고사,0213,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
60,CGV 목포평화광장,0280,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09
61,CGV 순천신대,0268,type4,쿵푸팬더4,TTT,2024.04.08 ~ 2024.05.09


# LotteCinema

In [59]:
def get_LC_EventList():
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    # 특별기획전 "EventClassificationCode":"10"
    body = {
        "ParamList": '{"MethodName":"GetEventLists","channelType":"","osType":"","osVersion":"","EventClassificationCode":"20","SearchText":"","CinemaID":"","PageNo":1,"PageSize":100,"MemberNo":"0"}'
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    event_df = pd.DataFrame(loads["Items"])

    goods_info = [get_LC_GoodsInfo(x) for x in event_df['EventID']]
    event_df = pd.concat([event_df[["EventID", "EventName", "ProgressStartDate", "ProgressEndDate"]], 
                           pd.json_normalize(goods_info)[["FrGiftID", "FrGiftNm"]]],
                         axis=1)

    event_df = event_df[~event_df['FrGiftID'].isna()].reset_index(drop=True)

    p = re.compile(r'\<([\d\D]+)\>') # 영화이름 추출 정규식
    event_df["movie_name"] = [
        '' if p.match(x) is None else p.match(x).group()[1:-1] 
        for x in event_df['EventName']
    ]

    p = re.compile(r'\,([\d\D]+)\)') # 굿즈이름 추출 정규식
    event_df["goods_name"] = [
        '' if len(p.findall(x))==0 else p.findall(x)[0].replace(' ', '')
        for x in event_df['FrGiftNm']
    ]
    return event_df

def get_LC_GoodsInfo(event_idx):
    # EventID, FrGiftID, FrGiftNm
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (event_idx)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    goods_info = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]
    if len(goods_info) == 0: 
        goods_info = None
    else:
        goods_info = goods_info[0]
    return goods_info

In [60]:
event_df = get_LC_EventList()
event_df

,EventID,EventName,ProgressStartDate,ProgressEndDate,FrGiftID,FrGiftNm,movie_name,goods_name
0,201010048024045,<목소리의 형태> 1주차 특전 스페셜 아트카드 증정 이벤트,2024.05.09,2024.05.15,9270,<목소리의 형태> 스페셜 아트카드,목소리의 형태,
1,201010016924208,<혹성탈출: 새로운 시대>시그니처 아트카드,2024.05.08,2024.05.14,9298,"<혹성탈출: 새로운 시대>시그니처 아트카드(5/8수~5/14화,회차당 수령제한O)",혹성탈출: 새로운 시대,회차당수령제한O
2,201010016924209,<포켓몬스터:성도지방>2주차 현장증정이벤트,2024.05.08,2024.05.14,9308,"<포켓몬스터:성도지방>2주차 굿즈(5/8수~5/14화,포스터세트)",포켓몬스터:성도지방,포스터세트
3,201010016924210,<미지수>1주차 현장증정이벤트,2024.05.08,2024.05.14,9311,"<미지수>1주차 굿즈(5/8수~5/14화,메인포스터)",미지수,메인포스터
4,201010016924211,<실바니안>2주차 현장증정이벤트,2024.05.08,2024.05.14,9312,"<실바니안>2주차 굿즈(5/8수~5/14화, 보송 스티커)",실바니안,보송스티커
5,201010016924206,<실바니안> 주말현장증정이벤트,2024.05.03,2024.05.09,9272,"<실바니안>주말굿즈(5/3금~5/9목,토끼머리띠)",실바니안,토끼머리띠
6,201010016924207,<범죄도시4>2주차 현장증정이벤트,2024.05.01,2024.05.07,9273,"<범죄도시4>2주차 굿즈(5/1수~5/7화, 친필엽서세트)",범죄도시4,친필엽서세트
7,201010016924203,<신차원!짱구는못말려> 어린이날 현장증정이벤트,2024.05.01,2024.05.07,9258,"<신차원!짱구는못말려>굿즈(5/1수~5/7화,캔뱃지)",신차원!짱구는못말려,캔뱃지
8,201010016924204,<실바니안>1주차 현장증정이벤트,2024.05.01,2024.05.07,9264,"<실바니안>1주차 굿즈(5/1수~5/7화, 토끼인형)",실바니안,토끼인형
9,201010016924205,<차이콥스키의 아내>1주차 현장증정이벤트,2024.05.01,2024.05.07,9271,"<차이콥스키의 아내>1주차 굿즈(5/1수~5/7화,티저포스터)",차이콥스키의 아내,티저포스터


In [61]:
def get_LC_Theaters(EventID, FrGiftID):
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    Theaters = pd.DataFrame(loads["CinemaDivisionGoods"])
    return Theaters

In [62]:
EventID = "201010016924197"
FrGiftID = "9190"
Theaters = get_LC_Theaters(EventID, FrGiftID)
Theaters

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,1,0001,1004,건대입구,Konkuk Univ,6,20,서울,Seoul
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,20,서울,Seoul
2,1,0001,1003,노원,Nowon,8,49,서울,Seoul
3,1,0001,1007,신림,Sillim,18,50,서울,Seoul
4,1,0001,1002,영등포,Yeongdeungpo,20,50,서울,Seoul
5,1,0001,1016,월드타워,WorldTower,22,50,서울,Seoul
6,1,0001,1008,청량리,Cheongnyangni,25,40,서울,Seoul
7,1,0002,3008,부평역사,Bupyeong Sation,16,50,경기/인천,Gyeonggi/Incheon
8,1,0002,3018,평촌(범계역),Pyeongchon,46,50,경기/인천,Gyeonggi/Incheon
9,1,0004,6004,수완(아울렛),Suwan,71,50,전라/광주,Cholla/Gwangju


In [63]:
def Get_LC_GoodsStatus(movie_filter="", goods_filter=""):
    event_df = get_LC_EventList()
    event_df = event_df[
        (event_df["EventName"].str.contains(movie_filter)) &
        (event_df["FrGiftNm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        EventID = event_df.loc[idx]["EventID"]
        FrGiftID = event_df.loc[idx]["FrGiftID"]
        Theaters = get_LC_Theaters(EventID, FrGiftID)
        Theaters["movie"] =  event_df.loc[idx]["movie_name"]
        Theaters["goods"] =  event_df.loc[idx]["goods_name"]
        Theaters["period"] =  event_df.loc[idx]["ProgressStartDate"] + '~' + event_df.loc[idx]["ProgressEndDate"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [64]:
Get_LC_GoodsStatus(goods_filter='엽서')

,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS,movie,goods,period
0,1,0001,1004,건대입구,Konkuk Univ,6,10,서울,Seoul,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
1,1,0001,1009,김포공항,Gimpo Int'l Airport,7,50,서울,Seoul,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
2,1,0001,1016,월드타워,WorldTower,22,0,서울,Seoul,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
3,1,0002,3024,수원(수원역),Suwon,23,10,경기/인천,Gyeonggi/Incheon,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
4,1,0003,4008,대전센트럴,Daejeon Central,5,20,충청/대전,Chungcheong/Daejeon,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
5,1,0004,6002,전주(백화점),Jeonju,109,40,전라/광주,Cholla/Gwangju,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
6,1,0005,5006,대구율하,Daegu Yulha,6,49,경북/대구,Gyeongbuk/Daegu,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
7,1,0101,2009,광복,Gwangbok,3,50,경남/부산/울산,Gyeongnam/Busan/Ulsan,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
8,1,0101,2004,부산본점,Busan,11,50,경남/부산/울산,Gyeongnam/Busan/Ulsan,범죄도시4,친필엽서세트,2024.05.01~2024.05.07
9,1,0101,5001,울산(백화점),Ulsan,19,50,경남/부산/울산,Gyeongnam/Busan/Ulsan,범죄도시4,친필엽서세트,2024.05.01~2024.05.07


#### 번외 수원 기획전 

In [65]:
EventID = "101010014224011"
url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
body = {
    "ParamList": '{"MethodName":"GetInfomationDeliveryEventDetail","channelType":"","osType":"","osVersion":"","EventID":%s}' % (EventID)
}
response = requests.post(url, data=body)
text = response.text
loads = json.loads(text)
suwon_list = loads["InfomationDeliveryEventDetail"][0]["GoodsGiftItems"]

suwon_list2 = []
for suwon in suwon_list:
    EventID = suwon["EventID"]
    FrGiftID = suwon["FrGiftID"]
    url = "https://event.lottecinema.co.kr/LCWS/Event/EventData.aspx"
    body = {
        "ParamList": '{"MethodName":"GetCinemaGoods","channelType":"","osType":"","osVersion":"","EventID":%s, "GiftID":%s}' % (EventID, FrGiftID)
    }
    response = requests.post(url, data=body)
    text = response.text
    loads = json.loads(text)
    suwon.update(loads["CinemaDivisionGoods"][0])
    suwon_list2.append(suwon)
    #     print(loads["CinemaDivisionGoods"])
pd.DataFrame(suwon_list2)

,EventID,FrGiftID,FrGiftNm,DivisionCode,DetailDivisionCode,CinemaID,CinemaNameKR,CinemaNameUS,SortSequence,Cnt,DetailDivisionNameKR,DetailDivisionNameUS
0,101010014224011,9183,수원 특별 기획전(스파이더맨 어크로스 더 유니버스),1,0002,3024,수원(수원역),Suwon,23,50,경기/인천,Gyeonggi/Incheon
1,101010014224011,9184,수원 특별 기획전(스파이더맨 노웨이 홈 FUN),1,0002,3024,수원(수원역),Suwon,23,100,경기/인천,Gyeonggi/Incheon
2,101010014224011,9185,수원 특별 기획전(그란투리스모),1,0002,3024,수원(수원역),Suwon,23,350,경기/인천,Gyeonggi/Incheon
3,101010014224011,9186,수원 특별 기획전(탑건: 매버릭),1,0002,3024,수원(수원역),Suwon,23,850,경기/인천,Gyeonggi/Incheon
4,101010014224011,9187,수원 특별 기획전(언어의 정원),1,0002,3024,수원(수원역),Suwon,23,350,경기/인천,Gyeonggi/Incheon
5,101010014224011,9188,수원 특별 기획전(날씨의 아이),1,0002,3024,수원(수원역),Suwon,23,100,경기/인천,Gyeonggi/Incheon


# Megabox

In [71]:
def get_MB_EventList():
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/eventMngDiv.do"
    body = {
        "currentPage": "1",
        "recordCountPerPage": "1000",
        "eventStatCd": "ONG",
        "eventTitle": "",
        "eventDivCd": "CED03",
        "eventTyCd": "",
        "orderReqCd": "ONGlist"
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, json=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)

    eventBtn_list = soup.find_all("a", class_="eventBtn")
    event_list = [
        { 
            "event_no": x["data-no"],
            "event_name": x["title"],
            "period": x.find("p", class_="date").get_text().replace("\t", "").replace("\n", "")
        }
        for x in eventBtn_list
    ]
    event_df = pd.DataFrame(event_list)
    goods_info = [get_MB_GoodsInfo(x) for x in event_df['event_no']]
    event_df = pd.concat([event_df, pd.json_normalize(goods_info)], axis=1)
    return event_df


def get_MB_GoodsInfo(event_idx):
    url = f"https://www.megabox.co.kr/event/detail?eventNo={event_idx}"
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    goods_tag = soup.find("button", attrs={"id":"btnSelectGoodsStock"})
    if goods_tag is None:
        goods_info = None
    else:
        goods_info = {
            "data-nm": goods_tag["data-nm"],
            "data-pn": goods_tag["data-pn"]
        }
    return goods_info

In [81]:
event_df = get_MB_EventList()
event_df[~event_df['data-pn'].isna()].sort_values('data-pn')

,event_no,event_name,period,data-nm,data-pn
14,15264,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,2024.04.10 ~ 2024.05.08,쿵푸팬더4 오리지널 티켓,FG000128
11,15275,메가박스 오리지널 티켓 No.108 <골드핑거> 상세보기,2024.04.13 ~ 2024.05.12,골드핑거 오리지널 티켓,FG000132
9,15325,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,2024.04.24 ~ 2024.05.24,범죄도시4 오리지널 티켓,FG000137


In [94]:
def get_MB_Theaters(goodsNo):
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/selectGoodsStockPrco.do"
    body = {
        "goodsNo": goodsNo
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, data=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    brch_list = soup.find_all("li", class_="brch")
    theater_list = [ 
        { 
            "theater": x.find("a").get_text(),
            "status": x.find("span").get_text()        
        }
        for x in brch_list
    ]
    Theaters = pd.DataFrame(theater_list)
    return Theaters

In [98]:
def Get_MB_GoodsStatus(goods_filter=""):
    event_df = get_MB_EventList()
    event_df = event_df[~event_df['data-pn'].isna()].sort_values('data-pn')
    event_df = event_df[
        (event_df["data-nm"].str.contains(goods_filter))
    ]
    GoodsStatus = pd.DataFrame()
    for idx in event_df.index:
        goodsNo = event_df.loc[idx]["data-pn"]
        Theaters = get_MB_Theaters(goodsNo)
        Theaters["event_name"] =  event_df.loc[idx]["event_name"]
        Theaters["data-nm"] =  event_df.loc[idx]["data-nm"]
        Theaters["period"] =  event_df.loc[idx]["period"]
        GoodsStatus = pd.concat([GoodsStatus, Theaters], ignore_index=True)
    return GoodsStatus

In [99]:
Get_MB_GoodsStatus()

,theater,status,event_name,data-nm,period
0,강남,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
1,강남대로(씨티)_영업종료,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
2,강동,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
3,군자,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
4,더 부티크 목동현대백화점,소진,메가박스 오리지널 티켓 No.107 <쿵푸팬더4> 상세보기,쿵푸팬더4 오리지널 티켓,2024.04.10 ~ 2024.05.08
...,...,...,...,...,...
308,원주혁신,소진,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24
309,춘천석사,소진,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24
310,제주삼화,소량보유,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24
311,제주서귀포,보유,메가박스 오리지널 티켓 No.109 <범죄도시4> 상세보기,범죄도시4 오리지널 티켓,2024.04.24 ~ 2024.05.24


In [84]:
# 이벤트 목록에 없는 굿즈 정보 확인

for i in range(120, 145):
    
    goodsNo = "FG000"+str(i)
a
    url = "https://www.megabox.co.kr/on/oh/ohe/Event/selectGoodsStockPrco.do"
    body = {
        "goodsNo": goodsNo
    }
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random
    }
    response = requests.post(url, data=body, headers=headers)
    text = response.text
    soup = BeautifulSoup(text)
    # brch_list = soup.find_all("li", class_="brch")
    # theater_list = [ 
    #     { 
    #         "theater": x.find("a").get_text(),
    #         "status": x.find("span").get_text()        
    #     }
    #     for x in brch_list
    # ]

    print(goodsNo, soup.find("div", class_="tit").get_text())
    # theater_list

FG000120 비키퍼 Dolby 포스터
FG000121 영웅본색 포스터
FG000122 기동전사 건담 SEED FREEDOM Dolby 포스터
FG000123 기동전사 건담 SEED FREEDOM MX4D 포스터
FG000124 남은 인생 10년 드로잉 카드
FG000125 오멘: 저주의 시작 A3 포스터
FG000126 쿵푸팬더4 Dolby 포스터
FG000127 로봇 드림 드로잉 카드
FG000128 쿵푸팬더4 오리지널 티켓
FG000129 8월의 크리스마스 인화사진
FG000130 
FG000131 
FG000132 골드핑거 오리지널 티켓
FG000133 듄 Dolby 포스터
FG000134 듄 MX4D 포스터
FG000135 고스트버스터즈 드로잉카드
FG000136 레옹 드로잉카드
FG000137 범죄도시4 오리지널 티켓
FG000138 혹성탈출 새로운 시대 Dolby 포스터
FG000139 쇼생크 탈출 오리지널티켓
FG000140 
FG000141 
FG000142 
FG000143 
FG000144 
